In [10]:
%run ../scripts/imports.py

data_path = open("../path_data.txt", 'r').readline()

Succesfully imported libraries and modules



In [87]:
patients = [load_patient(i, data_path, load_pac=False) for i in range(1, 10)]

Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient1
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient2
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient3
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient4
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient5
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient6
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient7
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient8
Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient9


In [83]:
df = pd.read_csv("../data/clinic_updated.csv", sep=',')
df

,subject,score,aff_hemisphere,symptom,day,state,side,inter
0,Kamalova,42,both,UPDRS3,1,OFF,no_side,Kamalova.1.both.OFF
1,Kamalova,24,both,UPDRS3,1,ON,no_side,Kamalova.1.both.ON
2,Kamalova,7,less_affected,Bradykinesia,1,OFF,right,Kamalova.1.less_affected.OFF
3,Kamalova,10,more_affected,Bradykinesia,1,OFF,left,Kamalova.1.more_affected.OFF
4,Kamalova,4,less_affected,Rigidity,1,OFF,right,Kamalova.1.less_affected.OFF
...,...,...,...,...,...,...,...,...
233,Akhmetshin,3,more_affected,Bradykinesia,5,ON,left,Akhmetshin.5.more_affected.ON
234,Akhmetshin,1,less_affected,Rigidity,5,ON,right,Akhmetshin.5.less_affected.ON
235,Akhmetshin,0,more_affected,Rigidity,5,ON,left,Akhmetshin.5.more_affected.ON
236,Akhmetshin,0,less_affected,Tremor,5,ON,right,Akhmetshin.5.less_affected.ON


In [84]:
patient_names_dict = {"Gavrilina": "Patient1", 
                 "Kamalova": "Patient2", 
                 "Konovalov": "Patient3", 
                 "Krasova": "Patient4", 
                 "Suhih": "Patient5", 
                 "Otroda": "Patient6", 
                 "Akhmetshin": "Patient7", 
                 "Boyko": "Patient8", 
                 "Kulagina": "Patient9"}


df.subject = df.subject.apply(lambda name: patient_names_dict[name])
df

,subject,score,aff_hemisphere,symptom,day,state,side,inter
0,Patient2,42,both,UPDRS3,1,OFF,no_side,Kamalova.1.both.OFF
1,Patient2,24,both,UPDRS3,1,ON,no_side,Kamalova.1.both.ON
2,Patient2,7,less_affected,Bradykinesia,1,OFF,right,Kamalova.1.less_affected.OFF
3,Patient2,10,more_affected,Bradykinesia,1,OFF,left,Kamalova.1.more_affected.OFF
4,Patient2,4,less_affected,Rigidity,1,OFF,right,Kamalova.1.less_affected.OFF
...,...,...,...,...,...,...,...,...
233,Patient7,3,more_affected,Bradykinesia,5,ON,left,Akhmetshin.5.more_affected.ON
234,Patient7,1,less_affected,Rigidity,5,ON,right,Akhmetshin.5.less_affected.ON
235,Patient7,0,more_affected,Rigidity,5,ON,left,Akhmetshin.5.more_affected.ON
236,Patient7,0,less_affected,Tremor,5,ON,right,Akhmetshin.5.less_affected.ON


In [85]:
df = df.drop(columns=['aff_hemisphere', 'inter'])
print(np.unique(df.symptom))
df_all = df.copy()
df_all

['Bradykinesia' 'Rigidity' 'Tremor' 'UPDRS3']


,subject,score,symptom,day,state,side
0,Patient2,42,UPDRS3,1,OFF,no_side
1,Patient2,24,UPDRS3,1,ON,no_side
2,Patient2,7,Bradykinesia,1,OFF,right
3,Patient2,10,Bradykinesia,1,OFF,left
4,Patient2,4,Rigidity,1,OFF,right
...,...,...,...,...,...,...
233,Patient7,3,Bradykinesia,5,ON,left
234,Patient7,1,Rigidity,5,ON,right
235,Patient7,0,Rigidity,5,ON,left
236,Patient7,0,Tremor,5,ON,right


### Converting clinics from Ksenia's dataframe to the usable patient.clinics table

In [94]:
# let's take subdf for one patient and try to convert it to df_new format as below

conditions_rest = ["1Day OFF Rest 180sec", "1Day ON Rest 180sec", "5Day OFF Rest 180sec", "5Day ON Rest 180sec"]
clinics_keys = ["r_brady", "r_rigid", "r_tremor", "l_brady", "l_rigid", "l_tremor"]
symptom_dict = {"Rigidity": "rigid", "Bradykinesia": "brady", "Tremor": "tremor"}




for patient in patients:
    print("\n" + patient.name + "\n")
    conditions = [c for c in conditions_rest if c in patient.conditions]
    print(*conditions, sep=',', end='\n\n')
    df_clinics = pd.DataFrame(columns=clinics_keys, index=conditions)
    df_patient = df_all[df_all.subject == patient.name]
    for i, row in df_patient.iterrows():
        if row.symptom == "UPDRS3": continue
        condition = create_condition_name(str(row.day) + "Day", row.state, "Rest 180sec")
        side = row.side[0]
        symptom = symptom_dict[row.symptom]
        col = side + "_" + symptom
        df_clinics.loc[condition, col] = row.score
    patient.clinics = df_clinics
    print(df_clinics)
    patient.save()


Patient1

1Day OFF Rest 180sec,1Day ON Rest 180sec,5Day OFF Rest 180sec,5Day ON Rest 180sec

                     r_brady r_rigid r_tremor l_brady l_rigid l_tremor
1Day OFF Rest 180sec       8       5        3       8       6        2
1Day ON Rest 180sec        4       1        0       3       2        0
5Day OFF Rest 180sec      13       7        3       9       5        2
5Day ON Rest 180sec        2       2        0       2       2        0
Saving Patient1 object to C:\Users\aleks\Downloads\data_pac\Patient1\Patient1.pkl ...
Pickling Patient1 without ['pac']
Done, 11.420416593551636 sec
File size: 2788.232021331787 MB
Returning filepath for saved file

Patient2

1Day OFF Rest 180sec,1Day ON Rest 180sec,5Day OFF Rest 180sec,5Day ON Rest 180sec

                     r_brady r_rigid r_tremor l_brady l_rigid l_tremor
1Day OFF Rest 180sec       7       4        0      10       6        0
1Day ON Rest 180sec        2       2        0       6       4        0
5Day OFF Rest 180sec       5 

### Old example xls

In [34]:
patient = load_patient(2, data_path, load_pac=False)

df = pd.read_excel(os.path.join(patient.root_dir, f"{patient.name}_clinics.xls"), index_col=0)

df

Patient directory:  C:\Users\aleks\Downloads\data_pac\Patient2


,off-U3Right.Bradyk,off-u3Left.Bradyk,off-u3Right.Rigidity,off-u3Left.Rigidity,off-u3Right.Tremor,off-u3Left.Tremor,on-u3Right.Bradykin,on-u3Left.Bradykin,on-u3Right.Rigidity,on-u3Left.Rigidity,on-u3Right.Tremor,on-u3Left.Tremor
Oper,9,12,5,7,1,0,1,5,2,2,0,0
5Day,5,6,4,6,0,0,0,1,1,1,0,0


In [35]:
conditions_rest = ["1Day OFF Rest 180sec", "1Day ON Rest 180sec", "5Day OFF Rest 180sec", "5Day ON Rest 180sec"]

conditions = [c for c in conditions_rest if c in patient.conditions]
clinics_keys = ["r_brady", "r_rigid", "r_tremor", "l_brady", "l_rigid", "l_tremor"]

df_new = pd.DataFrame(columns=clinics_keys, index=conditions)

clinics = defaultdict(dict)

for column in list(df.columns):
    ldopa = "OFF" if 'off' in column else 'ON'
    for day in list(df.index):
        if 'Day' not in day:
            continue
        condition = create_condition_name(day, ldopa, 'Rest 180sec')
        # side
        side = 'r' if "Right" in column else 'l'
        # symptom
        if "Brady" in column:
            key = side + '_brady'
        if "Rigid" in column:
            key = side + '_rigid'
        if "Tremor" in column:
            key = side + '_tremor'
        value = df.loc[day][column]
        clinics[condition][key] = value
        df_new.loc[condition][key] = value
        
        

In [36]:
df_new

,r_brady,r_rigid,r_tremor,l_brady,l_rigid,l_tremor
1Day OFF Rest 180sec,NaN,NaN,NaN,NaN,NaN,NaN
1Day ON Rest 180sec,NaN,NaN,NaN,NaN,NaN,NaN
5Day OFF Rest 180sec,5,4,0,6,6,0
5Day ON Rest 180sec,0,1,0,1,1,0
